#En esta lección, vamos a trabajar con una API (Interfaz de Programación de Aplicaciones) del clima para obtener las condiciones actuales en cualquier ubicación.

Obteniendo datos climáticos de una API
Vamos a utilizar una API para obtener datos climáticos en tiempo real de https://openweathermap.org.

La mayoría de las APIs requieren una clave de API. Podes pensar en esto como una contraseña única para vos.

Podes visitar el sitio web de openweathermap para obtener información sobre cómo configurar una clave de API para usar fuera de esta plataforma de aprendizaje.

Ejecuta la siguiente celda para configurar la clave de API para esta lección:

In [ ]:
from google.colab import userdata

In [ ]:
import requests
from IPython.display import Markdown

In [ ]:
# Almacena el valor de latitud en la variable 'lat'

lat = -34.6037  # Buenos Aires, Argentina

# Almacena el valor de longitud en la variable 'lon'
lon = -58.3816

# Almacena el valor de tu clave unica

api_key = userdata.get("OPEN_WEATHER_API_KEY")

Llamomos a la API del clima, pasando la clave de API y tus coordenadas:

Estos se agregan como parte de la cadena f url a continuación:

In [ ]:
url = f"https://api.openweathermap.org/data/2.5/forecast?units=metric&cnt=1&lat={lat}&lon={lon}&appid={api_key}"

# Usa la función get de la biblioteca requests para almacenar la respuesta de la API
response = requests.get(url)

Ahora extraemos los datos de la variable de respuesta:

Nota Las respuestas que obtenemos de las APIs generalmente se dan en formato JSON (JavaScript Object Notation), que es una variación de un diccionario de Python.

In [ ]:
# Toma la respuesta de la API (en JSON) y asígnala a un diccionario de Python
data = response.json()

# Imprime
print(data)

## Mostrar el pronóstico del tiempo

Para mostrar el pronóstico del tiempo de una manera bien formateada, necesitamos extraer información del diccionario de datos.

In [ ]:
temperatura = data['list'][0]['main']['temp']
descripcion = data['list'][0]['weather'][0]['description']
velocidad_viento = data['list'][0]['wind']['speed']

¡Imprimimos los resultados de manera agradable!

In [ ]:
print(f"Temperatura: {temperatura}")
print(f"Descripción del clima: {descripcion}")
print(f"Velocidad del viento: {velocidad_viento}")

Escribimos mejor un informe meteorológico:

In [ ]:
informe_clima = f"""La temperatura es {temperatura}°C.
Actualmente está {descripcion},
con una velocidad del viento de {velocidad_viento}m/s.
"""

print(informe_clima)

##Uso de un LLM para planificar

Vamos a expandir nuestro uso del LLM para obtener recomendaciones más relevantes para la agricultura. Modificaremos nuestro prompt para solicitar consejos sobre actividades agrícolas basadas en el clima:

In [ ]:
!pip install anthropic -q

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))

In [ ]:
def obtener_respuesta_ia(prompt):
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text

In [ ]:
prompt = f"""Basado en el siguiente pronóstico del tiempo para una región agrícola en Argentina,
proporciona recomendaciones sobre:
1. Si es un buen día para fumigar cultivos.
2. Si se deben regar los cultivos hoy.
3. Qué precauciones deben tomar los trabajadores agrícolas.
4. Cualquier impacto potencial en los cultivos.

Pronóstico: {informe_clima}
"""

In [ ]:
respuesta = obtener_respuesta_ia(prompt)
display(Markdown(respuesta))

##Visualización de datos climáticos
Para una mejor comprensión de los datos climáticos, vamos a crear una visualización simple usando matplotlib. Primero, necesitamos instalar (en caso de que no este) e importar la biblioteca:

In [ ]:
import matplotlib.pyplot as plt

Ahora, vamos a crear un gráfico de barras simple con los datos del clima:

In [ ]:
# Extrae más datos del pronóstico
humedad = data['list'][0]['main']['humidity']
presion = data['list'][0]['main']['pressure']

# Crea el gráfico
fig, ax = plt.subplots(figsize=(10, 6))

parametros = ['Temperatura (°C)', 'Humedad (%)', 'Velocidad del viento (m/s)', 'Presión (hPa)']
valores = [temperatura, humedad, velocidad_viento, presion]

ax.bar(parametros, valores, color=['red', 'blue', 'green', 'orange'])
ax.set_ylabel('Valor')
ax.set_title('Condiciones Climáticas Actuales')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Pronóstico a varios días
Para la planificación agrícola, es crucial tener un pronóstico de varios días. Vamos a modificar nuestra solicitud de API para obtener un pronóstico de 5 días:

In [ ]:
url_5dias = f"https://api.openweathermap.org/data/2.5/forecast?units=metric&lat={lat}&lon={lon}&appid={api_key}"
response_5dias = requests.get(url_5dias)
data_5dias = response_5dias.json()

# Extraer temperaturas y fechas
fechas = []
temperaturas = []

for pronostico in data_5dias['list']:
    fecha = pronostico['dt_txt']
    temp = pronostico['main']['temp']
    fechas.append(fecha)
    temperaturas.append(temp)

# Crear gráfico de líneas
plt.figure(figsize=(12, 6))
plt.plot(fechas, temperaturas, marker='o')
plt.title('Pronóstico de Temperatura para los Próximos 5 Días')
plt.xlabel('Fecha y Hora')
plt.ylabel('Temperatura (°C)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

##Integración con IA para predicción de rendimiento de cultivos
Ahora, vamos a simular cómo estos datos climáticos podrían usarse como entrada para un modelo de IA que predice el rendimiento de cultivos. Nota: Este es un ejemplo simplificado y no un modelo real.

In [ ]:
def simular_prediccion_rendimiento(temperatura, humedad, velocidad_viento):
    # Este es un modelo simplificado para propósitos ilustrativos
    rendimiento_base = 100  # quintales por hectárea
    factor_temp = 1 + (temperatura - 20) * 0.01  # 20°C se considera óptimo
    factor_humedad = 1 + (humedad - 60) * 0.005  # 60% se considera óptimo
    factor_viento = 1 - velocidad_viento * 0.02  # menos viento es mejor

    rendimiento_predicho = rendimiento_base * factor_temp * factor_humedad * factor_viento
    return max(0, rendimiento_predicho)  # el rendimiento no puede ser negativo

rendimiento = simular_prediccion_rendimiento(temperatura, humedad, velocidad_viento)
print(f"Rendimiento predicho del cultivo: {rendimiento:.2f} quintales por hectárea")

##Comparación de regiones agrícolas
Vamos a obtener y comparar datos climáticos para diferentes regiones agrícolas de Argentina:

In [ ]:
regiones = {
    "Pampa Húmeda": (-33.7, -60.5),
    "Cuyo": (-32.9, -68.8),
    "NOA": (-26.8, -65.2),
    "NEA": (-27.5, -58.8),
    "Patagonia": (-41.8, -68.9)
}

def obtener_datos_clima(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?units=metric&lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    return data['main']['temp'], data['main']['humidity'], data['wind']['speed']

# Recopilar datos
datos_regiones = {}
for region, coords in regiones.items():
    temp, hum, viento = obtener_datos_clima(coords[0], coords[1])
    datos_regiones[region] = {'temp': temp, 'hum': hum, 'viento': viento}

# Visualizar comparación
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

regiones = list(datos_regiones.keys())
temps = [datos['temp'] for datos in datos_regiones.values()]
hums = [datos['hum'] for datos in datos_regiones.values()]
vientos = [datos['viento'] for datos in datos_regiones.values()]

ax1.bar(regiones, temps, color='red')
ax1.set_title('Temperatura por Región')
ax1.set_ylabel('Temperatura (°C)')

ax2.bar(regiones, hums, color='blue')
ax2.set_title('Humedad por Región')
ax2.set_ylabel('Humedad (%)')

ax3.bar(regiones, vientos, color='green')
ax3.set_title('Velocidad del Viento por Región')
ax3.set_ylabel('Velocidad del Viento (m/s)')

plt.tight_layout()
plt.show()

In [ ]:
# Analizar con LLM
analisis_prompt = f"""Basado en los siguientes datos climáticos para diferentes regiones agrícolas de Argentina:

{datos_regiones}

Por favor, proporciona un análisis comparativo de las condiciones agrícolas en cada región, incluyendo:
1. Qué cultivos podrían ser más adecuados para cada región según estas condiciones.
2. Posibles desafíos o ventajas para la agricultura en cada región.
3. Recomendaciones generales para los agricultores en cada zona.
"""

respuesta = obtener_respuesta_ia(analisis_prompt)
display(Markdown(respuesta))